In [12]:
import numpy as np
from fastai import *
from fastai.vision import *
import torch
import os
from torchvision import transforms

**data from https://www.kaggle.com/mloey1/ahcd1**

In [13]:
data_path = Path('data')

In [14]:
test_x = np.genfromtxt(data_path/'csvTestImages 3360x1024.csv', delimiter=',')
test_y = np.genfromtxt(data_path/'csvTestLabel 3360x1.csv', delimiter=',')
test_y = test_y.astype(int) - 1
test_x = test_x.astype(int)


test_x.shape, test_y.shape, test_x[0].shape, test_y.max(), test_y.min(), max(test_x.max(axis=1))

((3360, 1024), (3360,), (1024,), 27, 0, 255)

#### Custom Dataset

In [15]:
class ArabicMnist(Dataset):
    def __init__(self, X, y=None):
        super().__init__()
        self.classes = np.unique(y)
        self.c = len(np.unique(y))
        self.X = X
        if y is not None: self.y = y

    def __getitem__(self, i):
        image = self.X[i]
        arr3D = np.tile(image[..., None], 3)
        arr3D = np.transpose(arr3D, (2, 1, 0))
        new_tensor = torch.from_numpy(arr3D).float()
        return (new_tensor, self.y[i])
    
    def __len__(self): return len(self.X)

In [16]:
test_x_reshaped = test_x.reshape([-1, 32, 32])

In [17]:
AM = ArabicMnist(test_x_reshaped[:3000], test_y[:3000])
AMV = ArabicMnist(test_x_reshaped[3000:], test_y[3000:])

In [18]:
data = DataBunch.create(AM, AMV, bs=2)

In [19]:
learn = create_cnn(data, models.resnet18, metrics=error_rate)

In [20]:
learn.loss_func

<function torch.nn.functional.nll_loss(input, target, weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='elementwise_mean')>

In [21]:
# %%debug
learn.fit(3, 1e-9)

Total time: 02:03
epoch  train_loss  valid_loss  error_rate
1      0.080469    -0.144593   0.975000    (00:42)
2      -0.110246   0.287240    0.961111    (00:40)
3      0.026357    -0.029954   0.961111    (00:40)



### MNSIT Fashion

In [36]:
train_df = pd.read_csv(str(data_path / 'fashion-mnist_train.csv'))
train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
number = 0
images, labels = [], []
def convert_to_numpy(row):
    global number
    label = row['label']
    pixels = []
    if number > 0:
        for i in range(1, 785):
            pixels.append(int(row['pixel' + str(i)]))
        pixels = np.array(pixels).reshape(28, 28)
        images.append(pixels)
        labels.append(label)
    number += 1
    return row

In [39]:
_ = train_df.apply(convert_to_numpy, axis=1)

In [40]:
len(images), len(labels)

(60000, 60000)

In [44]:
images = np.array(images)
labels = np.array(labels)

In [45]:
images.shape, labels.shape

((60000, 28, 28), (60000,))

In [46]:
AM = ArabicMnist(images[:30000], labels[:30000])
AMV = ArabicMnist(images[30000:], labels[30000:])

In [47]:
data = DataBunch.create(AM, AMV, bs=64)

In [48]:
data.c

10

In [49]:
learn = create_cnn(data, models.resnet18, metrics=error_rate)

In [50]:
learn.loss_func

<function torch.nn.functional.nll_loss(input, target, weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='elementwise_mean')>

In [51]:
# %%debug
learn.fit(3, 1e-9)

Total time: 01:10
epoch  train_loss  valid_loss  error_rate
1      -0.013104   -0.013139   0.895267    (00:24)
2      0.007585    -0.011116   0.895533    (00:23)
3      -0.004522   -0.012502   0.895933    (00:23)

